# 10. 그래프 이론

## 실전 문제

### 팀 결성 // RE(틀림)

#### 제한
* 풀이 시간 20분
* 시간 제한 2초
* 메모리 제한 128MB

##### 아이디어
* 서로소 집합 알고리즘의 find, union 기능을 함수로 만들어서 사용하자.

#### 시간 복잡도
* $O(N + {find 연산 개수}(1+\log_{2-{find 연산 개수}/N}N))$ 라고 한다. 정확한 계산은 조금 복잡할 듯 하다.
    * [참고할 만한 링크](https://hazel-developer.tistory.com/272)

#### 해설 본 후
* N, M의 범위가 모두 최대 100,000이므로 경로 압축을 이용한 서로소 집합 자료구조를 활용해야 함.
    * 경로 압축을 이용하지 않으면 시간 복잡도가 O(NM)이고, 이용하면 $O(N + {find 연산 개수}(1+\log_{2-{find 연산 개수}/N}N))$ 이므로 이 경우엔 경로 압축을 해야 시간 내에 해결 가능
* 기왕이면 가독성을 위해 함수를 앞에다 설정하고, 함수를 앞에다 설정하기 위해 팀을 기록할 리스트를 파라미터에 추가하자.
* 함수들을 잘못 짰다. find를 만들었으면 union에 이를 사용해야 하는데 그렇게 하지 않아 경로 압축이 안 됐을 것이다. 테스트 케이스에는 운좋게 통과가 되었다.
* 같은 팀 여부 확인 연산을 굳이 함수로 만들 필요 없이, find를 두 번 이용하여 바로 비교할 수 있다.

In [2]:
# 학생 번호 N, 연산 개수 M 입력
N, M = map(int, input().split())

# 학생들의 팀을 기록할 리스트 선언(자기 자신을 팀으로 초기화)
teams = [i for i in range(N+1)]

# 해당 학생의 팀을 찾는 함수 정의
def find_team(a):
    if teams[a] != a:
        teams[a] = find_team(teams[a])
    return teams[a]

# 팀 합치기 연산 정의
def union_team(a, b):
    team_a = teams[a]
    team_b = teams[b]
    
    if team_a < team_b:
        teams[team_b] = team_a
    else:
        teams[team_a] = team_b
    
# 같은 팀 여부 확인 연산 정의
def check_same_team(a, b):
    if teams[a] == teams[b]:
        return 'YES'
    else:
        return 'NO'

# 연산 조건 입력 받아서 진행
for _ in range(M):
    oper, a, b = map(int, input().split())
    
    # 합치기 연산
    if oper == 0:
        union_team(a, b)
    # 같은 팀 여부 확인 연산
    else:
        print(check_same_team(a, b))

NO
NO
YES


#### 해설 코드 참조

In [4]:
# 특정 원소가 속한 집합을 찾기
def find_team(teams, a):
    # 루트 노드가 아니라면, 루트 노드를 찾을 때까지 재귀적으로 호출
    if teams[a] != a:
        teams[a] = find_team(teams, teams[a])
    return teams[a]

# 두 원소가 속한 집합을 합치기
def union_team(teams, a, b):
    team_a = find_team(teams, a)
    team_b = find_team(teams, b)
    
    if team_a < team_b:
        teams[team_b] = team_a
    else:
        teams[team_a] = team_b

N, M = map(int, input().split())
teams = [i for i in range(N+1)] # 부모 테이블 초기화(부모를 자기 자신으로)
    
# 각 연산을 하나씩 확인
for _ in range(M):
    oper, a, b = map(int, input().split())
    # 합집합(union) 연산인 경우
    if oper == 0:
        union_team(teams, a, b)
    # 찾기(find) 연산인 경우
    elif oper == 1:
        if find_team(teams, a) == find_team(teams, b):
            print('YES')
        else:
            print('NO')

NO
NO
YES


### 도시 분할 계획
https://www.acmicpc.net/problem/1647

#### 제한
* 풀이 시간 40분
* 시간 제한 2초
* 메모리 제한 256MB

#### 아이디어
* 크루스칼 이용하여 최소 신장 트리 만든 다음, 가장 높은 비용의 간선 하나를 제거하면 될 것 같다.
* 시간 초과가 난다. 입력값이 최대 1,000,000개가 될 수 있으므로 sys.stdin.readline()을 이용해보자 => 됐다.

#### 시간 복잡도
* 크루스칼 알고리즘을 사용했고, 간선의 개수가 M개이므로 O(MlogM).
* M은 최대 1,000,000이므로 시간 내에 해결이 가능하다.

#### 해설 본 후
* 어차피 간선이 유지비 기준 오름차순으로 정렬되어 있으므로, 최대 유지비의 길을 저장할 때 max()를 쓸 필요 없이 해당 값으로 갱신만 해주면 된다.
* 답지에선 sys.stdin.readline()을 안 쓰고 그냥 input()으로 했는데, 백준에서 채점 돌려보면 input()으로 하면 시간초과가 난다.

In [14]:
import sys

# 어떤 집이 속한 마을을 확인하는 함수 선언
def find_group(groups, a):
    if groups[a] != a:
        groups[a] = find_group(groups, groups[a])
    return groups[a]

# 두 집이 속한 마을을 같은 마을로 합치는 함수 선언
def union_group(groups, a, b):
    group_a = find_group(groups, a)
    group_b = find_group(groups, b)
    
    if group_a < group_b:
        groups[group_b] = group_a
    else:
        groups[group_a] = group_b

# 집 개수 N, 길 개수 M 입력
N, M = map(int, input().split())

# 집들이 속한 마을 정보를 담는 리스트 선언(자신이 속한 마을을 자신의 마을로 초기화)
groups = [i for i in range(N+1)]

# 길의 정보 입력
roads = []
for _ in range(M):
    A, B, C = map(int, sys.stdin.readline().split())
    # A번 집에서 B번 집으로 가는 경로의 유지비가 C임.
    roads.append((C, (A, B)))

# 길을 유지비 기준 오름차순으로 정렬
roads.sort()

# 최소 신장 트리를 통해 구한 길의 전체 유지비와 길 중 가장 유지비가 높은 길의 유지비를 초기화
result = 0
max_cost = 0

# 모든 길을 확인하며
for cost, road in roads:
    # 길로 이어진 두 집의 속한 마을이 다르다면
    if find_group(groups, road[0]) != find_group(groups, road[1]):
        # 속한 마을을 합쳐주고
        union_group(groups, road[0], road[1])
        # 전체 유지비에 해당 길의 유지비를 추가하고
        result += cost
        # 지금까지 확인한 길보다 유지비가 비싸다면 최대 유지비를 가진 길을 갱신
        max_cost = max(max_cost, cost) ## 어차피 오름차순 정렬되어 있으므로 max()를 쓰지 않고 갱신해도 된다.

# 전체 유지비에서 가장 유지비가 큰 길을 빼서 마을 두 개를 생성할 수 있다.
print(result - max_cost)

8


### 커리큘럼

#### 제한
* 풀이 시간 50분
* 시간 제한 2초
* 메모리 제한 128MB

#### 아이디어
* 

#### 시간 복잡도
* 

#### 해설 본 후
* 